In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2, matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from sklearn.preprocessing import normalize
import math


import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

from torch.utils.data import Dataset, DataLoader
from torchtext import data
from torch.utils.data import TensorDataset, random_split


SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [2]:
train_data = pd.read_csv('siamese_data.csv')
train_data.head(2)

,posting_id_1,posting_id_2,title_1,title_2,image_1,image_2,label
0,train_129225211,train_2278313361,Paper Bag Victoria Secret,PAPER BAG VICTORIA SECRET,0000a68812bc7e98c42888dfb1c07da0.jpg,f83b49a86a0ee8592e3bf0204da3fbdf.jpg,1
1,train_129225211,train_866245802,Paper Bag Victoria Secret,BGSR Jaket Cagoule / Jaket Cagoule Premium / J...,0000a68812bc7e98c42888dfb1c07da0.jpg,5b9a1c01bf01447982a8f0a593909d07.jpg,0


In [3]:
TEXT_1 = data.Field(sequential=True,
  init_token='(sos)',  # start of sequence
  eos_token='(eos)',   # replace parens with less, greater
  lower=True,
  tokenize=data.utils.get_tokenizer("basic_english"))

TEXT_2 = data.Field(sequential=True,
  init_token='(sos)',  # start of sequence
  eos_token='(eos)',   # replace parens with less, greater
  lower=True,
  tokenize=data.utils.get_tokenizer("basic_english"))

LABEL =  data.LabelField(dtype = torch.float)

In [4]:
train = data.TabularDataset('siamese_data.csv',
                            format='csv',
                            fields=[('title_1',TEXT_1),('title_2',TEXT_2),('label',LABEL)])

# train

In [5]:
spacy_en = spacy.load('en_core_web_sm')

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

MAX_VOCAB_SIZE = 15_000

TEXT_1.build_vocab(train, max_size = MAX_VOCAB_SIZE)
TEXT_2.build_vocab(train, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)
# print(f"Unique tokens in TEXT vocabulary: {len(TEXT_1.vocab)}")

In [6]:
BATCH_SIZE = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator = BucketIterator.splits(train,batch_size = BATCH_SIZE,
                                       sort_key = lambda x: len(x.TEXT_1) ,device = device)

In [7]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [8]:
# class Decoder(nn.Module):
#     def __init__(self, output_dim, emb_dim, hid_dim, dropout):
#         super().__init__()

#         self.hid_dim = hid_dim
#         self.output_dim = output_dim

#         self.embedding = nn.Embedding(output_dim, emb_dim)

#         self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)

#         self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)

#         self.dropout = nn.Dropout(dropout)
    
#     def forward(self, input, hidden, context):
        
#         #input = [batch size]
#         #hidden = [n layers * n directions, batch size, hid dim]
#         #context = [n layers * n directions, batch size, hid dim]

#         #n layers and n directions in the decoder will both always be 1, therefore:
#         #hidden = [1, batch size, hid dim]
#         #context = [1, batch size, hid dim]

#         input = input.unsqueeze(0)

#         #input = [1, batch size]

#         embedded = self.dropout(self.embedding(input))

#         #embedded = [1, batch size, emb dim]

#         emb_con = torch.cat((embedded, context), dim = 2)

#         #emb_con = [1, batch size, emb dim + hid dim]

#         output, hidden = self.rnn(emb_con, hidden)

#         #output = [seq len, batch size, hid dim * n directions]
#         #hidden = [n layers * n directions, batch size, hid dim]

#         #seq len, n layers and n directions will always be 1 in the decoder, therefore:
#         #output = [1, batch size, hid dim]
#         #hidden = [1, batch size, hid dim]

#         output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
#                            dim = 1)

#         #output = [batch size, emb dim + hid dim * 2]

#         prediction = self.fc_out(output)

#         #prediction = [batch size, output dim]

#         return prediction, hidden

In [9]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.device = device

    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        

        return outputs

In [15]:
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(TEXT.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [16]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(11007, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(11007, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=11007, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 22,887,423 trainable parameters


In [18]:
optimizer = optim.Adam(model.parameters())

In [20]:
TRG_PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [56]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch
#         trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, src)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [57]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [58]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
#     valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
#     if valid_loss < best_valid_loss:
#         best_valid_loss = valid_loss
#         torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
#     print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

AttributeError: 'BucketIterator' object has no attribute 'shape'

In [43]:
for i, batch in enumerate(train_iterator):
    print(batch)
    break